## Математическая постановка задачи о динамическом поведении системы

### 1. Основные уравнения динамики

Система материальных точек описывается координатами:
$$
\mathbf{r}_{ij}(t) = \begin{pmatrix} x_{ij}(t) \\ y_{ij}(t) \\ z_{ij}(t) \end{pmatrix}, \quad i=1,\dots,N,\ j=1,\dots,M
$$

**Уравнение движения** для каждой точки:
$$
m\frac{d^2\mathbf{r}_{ij}}{dt^2} = \mathbf{F}_{ij}^{\text{total}}(\mathbf{r}, \mathbf{v}, t)
$$
где:
- $m = 0.1\ \text{кг}$ - масса точки
- $\mathbf{F}_{ij}^{\text{total}}$ - сумма всех сил

### 2. Составляющие сил

#### a) Гравитация:
$$
\mathbf{F}_{ij}^{\text{gravity}} = \begin{pmatrix} 0 \\ 0 \\ -mg \end{pmatrix},\quad g = 9.8\ \text{м/с}^2
$$

#### b) Взаимодействие со сферой:

Условие контакта:
$$
\|\mathbf{r}_{ij} - \mathbf{c}\| < R
$$
где $\mathbf{c} = (1.0, 1.0, 1.0)^T$, $R = 0.25$

Сила реакции:
$$
\mathbf{F}_{ij}^{\text{sphere}} = 
\begin{cases} 
C_s(R - \|\mathbf{r}\|)\frac{\mathbf{r}}{\|\mathbf{r}\|} + B_s\left(\mathbf{v}_{ij}\cdot\frac{\mathbf{r}}{\|\mathbf{r}\|}\right)\frac{\mathbf{r}}{\|\mathbf{r}\|}, & \|\mathbf{r}\| < R \\
\mathbf{0}, & \text{иначе}
\end{cases}
$$
где $\mathbf{r} = \mathbf{r}_{ij} - \mathbf{c}$

#### c) Нерастяжимые связи:

Ограничения для соседних точек:
$$
\|\mathbf{r}_{ij} - \mathbf{r}_{kl}\| = l_0,\quad l_0 = 0.1
$$

### 3. Численное интегрирование

**Метод Верле**:
$$
\mathbf{r}_{ij}(t+\Delta t) = 2\mathbf{r}_{ij}(t) - \mathbf{r}_{ij}(t-\Delta t) + \frac{\mathbf{F}_{ij}^{\text{total}}(t)}{m}\Delta t^2
$$
$$
\mathbf{v}_{ij}(t) = \frac{\mathbf{r}_{ij}(t+\Delta t) - \mathbf{r}_{ij}(t-\Delta t)}{2\Delta t}
$$

### 4. Проекционный метод

**Коррекция положения** для каждой связи:
$$
\Delta\mathbf{r} = \mathbf{r}_{kl} - \mathbf{r}_{ij}
$$
$$
\delta = \frac{\|\Delta\mathbf{r}\| - l_0}{2}
$$
$$
\mathbf{r}_{ij} \leftarrow \mathbf{r}_{ij} + \delta\frac{\Delta\mathbf{r}}{\|\Delta\mathbf{r}\|}
$$
$$
\mathbf{r}_{kl} \leftarrow \mathbf{r}_{kl} - \delta\frac{\Delta\mathbf{r}}{\|\Delta\mathbf{r}\|}
$$

### 5. Полная система

1. **Дифференциальные уравнения**:
$$
\frac{d^2\mathbf{r}}{dt^2} = \mathbf{M}^{-1}\mathbf{F}(\mathbf{r}, \mathbf{v}, t)
$$

2. **Алгебраические ограничения**:
$$
\|\mathbf{r}_{ij} - \mathbf{r}_{kl}\| - l_0 = 0
$$
$$
\|\mathbf{r}_{ij} - \mathbf{c}\| - R \geq 0
$$

3. **Начальные условия**:
$$
\mathbf{r}_{ij}(0) = \begin{pmatrix} il_0 \\ jl_0 \\ 1.5 \end{pmatrix},\quad \mathbf{v}_{ij}(0) = \mathbf{0}
$$

### 6. Алгоритм решения

1. Вычисление сил $\mathbf{F}^{\text{total}}$
2. Интегрирование методом Верле
3. Применение проекционного метода
4. Обработка столкновений
5. Коррекция скоростей

Эта постановка сочетает:
- Дискретную модель масс-пружин
- Жесткие ограничения через проекционный метод
- Реалистичное взаимодействие с препятствиями


## Общее описание кода

Данный код реализует физическую модель ткани как сетки материальных точек (осцилляторов), соединенных нерастяжимыми связями. Ткань взаимодействует со сферой, на которую она падает под действием силы тяжести. Модель включает:

1. Динамику материальных точек (второй закон Ньютона)
2. Обработку столкновений со сферой (пружинные и демпфирующие силы)
3. Проекционный метод для поддержания нерастяжимости связей
4. Визуализацию в 3D и 2D проекциях

## Математические модели и их реализация в коде

### 1. Параметры системы

```python
N = 20  # Число осцилляторов по горизонтали
M = 20  # Число осцилляторов по вертикали
m = 0.1  # Масса каждого осциллятора
g = 9.8  # Ускорение свободного падения
dt = 0.01  # Шаг времени
nstep = 1000  # Количество шагов
```

- Система представляет собой сетку N×M материальных точек
- Каждая точка имеет массу `m`
- На систему действует гравитация с ускорением `g`
- Интегрирование выполняется с шагом `dt` для `nstep` шагов

### 2. Параметры связей и сферы

```python
l0 = 0.1  # Длина нерастяжимой связи
Cs = 500.0  # Жесткость при контакте со сферой
Bs = 5.0  # Коэффициент демпфирования при контакте со сферой
R = 0.25  # Радиус сферы
sphere_center = np.array([1.0, 1.0, 1.0])  # Центр сферы
```

- Точки соединены связями длиной `l0` (нерастяжимыми)
- При контакте со сферой действуют:
  - Пружинная сила с жесткостью `Cs`
  - Демпфирующая сила с коэффициентом `Bs`
- Сфера имеет радиус `R` и центр в `sphere_center`

### 3. Инициализация положений и скоростей

```python
x = np.zeros((N, M, 3), float)  # Положения (x, y, z)
v = np.zeros((N, M, 3), float)  # Скорости (vx, vy, vz)
a = np.zeros((N, M, 3), float)  # Ускорения (ax, ay, az)

# Начальное расположение
for i in range(N):
    for j in range(M):
        x[i, j] = [i * l0 + 0.0, j * l0 + 0.0, 1.5]  # Начальная высота 1.5
```

- Точки инициализируются в прямоугольной сетке с шагом `l0`
- Начальная высота по z = 1.5 (выше сферы)

### 4. Метод Верле (Verlet) для интегрирования

Реализация в коде:
```python
x_next = 2 * x - x_prev + a * dt**2
v_new = (x_next - x_prev) / (2 * dt)
x_prev = x.copy()
x = x_next.copy()
v = v_new.copy()
```

### 5. Расчет сил

Основные силы в системе:
1. Гравитация: `F_gravity = -m * g` (по оси z)
2. Силы отталкивания от сферы:
   - Пружинная составляющая: `F_spring = Cs * penetration * normal`
   - Демпфирующая составляющая: `F_damping = Bs * relative_velocity * normal`

Реализация в функции `compute_forces`:
```python
def compute_forces(x, fixed):
    forces = np.zeros((N, M, 3), float)
    # Гравитация
    for i in range(N):
        for j in range(M):
            if not fixed[i, j]:
                forces[i, j, 2] -= m * g
    
    # Взаимодействие со сферой
    for i in range(N):
        for j in range(M):
            if fixed[i, j]:
                continue
            r = x[i, j] - sphere_center
            dist = np.linalg.norm(r)
            if dist < R:  # Контакт со сферой
                normal = r / dist
                penetration = R - dist
                relative_velocity = np.dot(v[i, j], normal)
                forces[i, j] += Cs * penetration * normal + Bs * relative_velocity * normal
                
                # Коррекция положения
                for _ in range(5):
                    r = x[i, j] - sphere_center
                    dist = np.linalg.norm(r)
                    if dist < R:
                        normal = r / dist
                        penetration = R - dist
                        x[i, j] += penetration * normal
                
                # Коррекция скорости (только тангенциальная составляющая)
                normal = r / dist
                v_normal = np.dot(v[i, j], normal) * normal
                v_tangent = v[i, j] - v_normal
                v[i, j] = v_tangent
```

### 6. Проекционный метод для нерастяжимых связей

Метод итеративно корректирует положения точек, чтобы длины связей были равны `l0`.

Уравнение коррекции:
```
correction = (l0 - dist) / (2 * dist) * dx
```

Реализация в функции `enforce_constraints`:
```python
def enforce_constraints(x):
    tolerance = 1e-6
    max_iterations = 100
    for _ in range(max_iterations):
        max_error = 0.0
        for i in range(N):
            for j in range(M):
                # Горизонтальные связи
                if i < N - 1:
                    dx = x[i + 1, j] - x[i, j]
                    dist = np.linalg.norm(dx)
                    error = abs(dist - l0)
                    max_error = max(max_error, error)
                    correction = (l0 - dist) / (2 * dist) * dx
                    x[i + 1, j] += correction
                    x[i, j] -= correction
                # Вертикальные связи
                if j < M - 1:
                    dy = x[i, j + 1] - x[i, j]
                    dist = np.linalg.norm(dy)
                    error = abs(dist - l0)
                    max_error = max(max_error, error)
                    correction = (l0 - dist) / (2 * dist) * dy
                    x[i, j + 1] += correction
                    x[i, j] -= correction
        if max_error < tolerance:
            break
```

### 7. Визуализация

Функция `visualize_step` создает 4 графика:
1. 3D вид всей системы
2. Вид сверху (XY плоскость)
3. Вид сбоку (YZ плоскость)
4. Вид сбоку (XZ плоскость)

Каждый график показывает:
- Положения точек
- Связи между ними
- Сферу (в 3D виде)

## Физические принципы и численные методы

1. **Динамика системы**:
   
- Уравнения движения решаются методом Верле (разновидность метода Верле-лета)
- Метод Верле является симлектическим, что хорошо сохраняет энергию

3. **Обработка столкновений со сферой**:

- Используется модель пружины-демпфера для контакта со сферой
- Применяется итеративная коррекция положения для точного нахождения на поверхности

4. **Нерастяжимые связи**:

- Проекционный метод (Position-Based Dynamics подход)
- Итеративно корректирует положения точек для соблюдения связей

5. **Устойчивость**:
   
- Малый шаг времени (dt равен или менее 0.01)
- Множество итераций проекционного метода (до 100), при необходимости


# ЗАДАНИЕ

- реализовать этот алгоритм средствами **TaiChi** с использованием **Pointer Memory**
- принять что моделируемый квадрат имеет размеры 1 м на 1 м
- провести моделирование систем осцилляторов
  1. сетка 5 на 5
  2. сетка 10 на 10
  3. сетка 20 на 20
  4. сетка 50 на 50
  5. сетка 100 на 100
- для каждой системы осцилляторов провести расчет 25 шагов по времени с обязательным измерением времени выполнения на **NumPy** и на **TaiChi**
- построить график зависимости времени выполнения от размерности сетки осцилляторов
- построить график зависимости ускорения относительно **NumPy** от размерности сетки осцилляторов
- для размерности сетки 50 на 50 построить анимацию процесса падения на шар для **NumPy** и для **TaiChi**, и сделать слайд с демонстрацией этих анимаций для сравнения

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os

# Параметры системы
N = 20  # Число осцилляторов по горизонтали
M = 20  # Число осцилляторов по вертикали
m = 0.1  # Масса каждого осциллятора
g = 9.8  # Ускорение свободного падения
dt = 0.01  # Шаг времени
nstep = 25  # Количество шагов

# Параметры связей
l0 = 0.1  # Длина нерастяжимой связи
Cs = 500.0  # Жесткость при контакте со сферой
Bs = 5.0  # Коэффициент демпфирования при контакте со сферой
R = 0.25  # Радиус сферы
sphere_center = np.array([1.0, 1.0, 1.0])  # Центр сферы

# Начальные условия
x = np.zeros((N, M, 3), float)  # Положения (x, y, z)
v = np.zeros((N, M, 3), float)  # Скорости (vx, vy, vz)
a = np.zeros((N, M, 3), float)  # Ускорения (ax, ay, az)

# Расположение осцилляторов в начальный момент
for i in range(N):
    for j in range(M):
        x[i, j] = [i * l0 + 0.0, j * l0 + 0.0, 1.5]  # Начальная высота 1.5

# Предыдущие положения для метода Верле
x_prev = x - v * dt + 0.5 * a * dt**2

# Массив для отслеживания фиксированных масс
fixed = np.zeros((N, M), dtype=bool)

# Функция для расчета сил
def compute_forces(x, fixed):
    forces = np.zeros((N, M, 3), float)  # Массив сил для всех масс
    # Гравитация
    for i in range(N):
        for j in range(M):
            if not fixed[i, j]:
                forces[i, j, 2] -= m * g
    # Взаимодействие со сферой
    for i in range(N):
        for j in range(M):
            if fixed[i, j]:  # Пропускаем фиксированные массы
                continue
            r = x[i, j] - sphere_center
            dist = np.linalg.norm(r)
            if dist < R:  # Если масса внутри сферы
                normal = r / dist  # Нормаль к поверхности сферы
                penetration = R - dist  # Глубина проникновения
                relative_velocity = np.dot(v[i, j], normal)  # Проекция скорости на нормаль
                # Добавляем силу отталкивания и демпфирования
                forces[i, j] += Cs * penetration * normal + Bs * relative_velocity * normal
                # Корректируем положение массы на поверхности сферы
                for _ in range(5):  # Итеративная коррекция
                    r = x[i, j] - sphere_center
                    dist = np.linalg.norm(r)
                    if dist < R:
                        normal = r / dist
                        penetration = R - dist
                        x[i, j] += penetration * normal
                # Разделяем скорость на нормальную и тангенциальную составляющие
                normal = r / dist
                v_normal = np.dot(v[i, j], normal) * normal
                v_tangent = v[i, j] - v_normal
                v[i, j] = v_tangent  # Сохраняем только тангенциальную составляющую
                # Проверка, что масса на поверхности
                assert np.isclose(np.linalg.norm(x[i, j] - sphere_center), R), \
                    f"Ошибка: масса не на поверхности сферы, dist={np.linalg.norm(x[i, j] - sphere_center)}"
    return forces

# Проекционный метод для восстановления нерастяжимых связей
def enforce_constraints(x):
    tolerance = 1e-6  # Допустимая погрешность
    max_iterations = 100  # Максимальное число итераций
    for _ in range(max_iterations):
        max_error = 0.0
        for i in range(N):
            for j in range(M):
                # Коррекция связей с соседями
                if i < N - 1:  # Горизонтальная связь 
                    dx = x[i + 1, j] - x[i, j]
                    dist = np.linalg.norm(dx)
                    error = abs(dist - l0)
                    max_error = max(max_error, error)
                    correction = (l0 - dist) / (2 * dist) * dx
                    x[i + 1, j] += correction
                    x[i, j] -= correction
                if j < M - 1:  # Вертикальная связь
                    dy = x[i, j + 1] - x[i, j]
                    dist = np.linalg.norm(dy)
                    error = abs(dist - l0)
                    max_error = max(max_error, error)
                    correction = (l0 - dist) / (2 * dist) * dy
                    x[i, j + 1] += correction
                    x[i, j] -= correction
        if max_error < tolerance:
            break

# Функция для визуализации текущего состояния
def visualize_step(x, step):
    xs, ys, zs = x[:, :, 0], x[:, :, 1], x[:, :, 2]
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))  # Четыре панели (2x2)
    # Общий трехмерный вид
    ax_general = fig.add_subplot(2, 2, 1, projection='3d')
    ax_general.scatter(xs, ys, zs, c='blue', s=50, label='Массы')
    # Отображение связей
    for i in range(N):
        for j in range(M):
            if i < N - 1:  # Горизонтальные связи
                ax_general.plot([x[i, j, 0], x[i + 1, j, 0]],
                                [x[i, j, 1], x[i + 1, j, 1]],
                                [x[i, j, 2], x[i + 1, j, 2]], color='gray', linewidth=0.5)
            if j < M - 1:  # Вертикальные связи
                ax_general.plot([x[i, j, 0], x[i, j + 1, 0]],
                                [x[i, j, 1], x[i, j + 1, 1]],
                                [x[i, j, 2], x[i, j + 1, 2]], color='gray', linewidth=0.5)
    # Отображение сферы
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 50)
    x_sphere = sphere_center[0] + R * np.outer(np.cos(u), np.sin(v))
    y_sphere = sphere_center[1] + R * np.outer(np.sin(u), np.sin(v))
    z_sphere = sphere_center[2] + R * np.outer(np.ones(np.size(u)), np.cos(v))
    ax_general.plot_surface(x_sphere, y_sphere, z_sphere, color='red', alpha=0.5, label='Сфера')
    # Настройка графика
    ax_general.set_xlim(-0.1, 2.0)
    ax_general.set_ylim(-0.1, 2.0)
    ax_general.set_zlim(-0.1, 2.0)
    ax_general.set_box_aspect([1, 1, 1])
    ax_general.set_xlabel('X')
    ax_general.set_ylabel('Y')
    ax_general.set_zlabel('Z')
    ax_general.set_title("Общий вид")
    ax_general.legend()
    # Вид сверху вдоль оси Z
    ax_top = fig.add_subplot(2, 2, 2)
    ax_top.scatter(xs, ys, c='blue', s=50, label='Массы')
    for i in range(N):
        for j in range(M):
            if i < N - 1:  # Горизонтальные связи
                ax_top.plot([x[i, j, 0], x[i + 1, j, 0]],
                            [x[i, j, 1], x[i + 1, j, 1]], color='gray', linewidth=0.5)
            if j < M - 1:  # Вертикальные связи
                ax_top.plot([x[i, j, 0], x[i, j + 1, 0]],
                            [x[i, j, 1], x[i, j + 1, 1]], color='gray', linewidth=0.5)
    ax_top.set_xlim(-0.1, 2.0)
    ax_top.set_ylim(-0.1, 2.0)
    ax_top.set_xlabel('X')
    ax_top.set_ylabel('Y')
    ax_top.set_title("Вид сверху (по Z)")
    ax_top.legend()
    # Вид сбоку вдоль оси X
    ax_side_x = fig.add_subplot(2, 2, 3)
    ax_side_x.scatter(ys, zs, c='blue', s=50, label='Массы')
    for i in range(N):
        for j in range(M):
            if i < N - 1:  # Горизонтальные связи
                ax_side_x.plot([x[i, j, 1], x[i + 1, j, 1]],
                               [x[i, j, 2], x[i + 1, j, 2]], color='gray', linewidth=0.5)
            if j < M - 1:  # Вертикальные связи
                ax_side_x.plot([x[i, j, 1], x[i, j + 1, 1]],
                               [x[i, j, 2], x[i, j + 1, 2]], color='gray', linewidth=0.5)
    ax_side_x.set_xlim(-0.1, 2.0)
    ax_side_x.set_ylim(-0.1, 2.0)
    ax_side_x.set_xlabel('Y')
    ax_side_x.set_ylabel('Z')
    ax_side_x.set_title("Вид сбоку (по X)")
    ax_side_x.legend()
    # Вид сбоку вдоль оси Y
    ax_side_y = fig.add_subplot(2, 2, 4)
    ax_side_y.scatter(xs, zs, c='blue', s=50, label='Массы')
    for i in range(N):
        for j in range(M):
            if i < N - 1:  # Горизонтальные связи
                ax_side_y.plot([x[i, j, 0], x[i + 1, j, 0]],
                               [x[i, j, 2], x[i + 1, j, 2]], color='gray', linewidth=0.5)
            if j < M - 1:  # Вертикальные связи
                ax_side_y.plot([x[i, j, 0], x[i, j + 1, 0]],
                               [x[i, j, 2], x[i, j + 1, 2]], color='gray', linewidth=0.5)
    ax_side_y.set_xlim(-0.1, 2.0)
    ax_side_y.set_ylim(-0.1, 2.0)
    ax_side_y.set_xlabel('X')
    ax_side_y.set_ylabel('Z')
    ax_side_y.set_title("Вид сбоку (по Y)")
    ax_side_y.legend()
    # Сохранение кадра
    frame_path = os.path.join(output_dir, f"frame_{step:04d}.png")
    plt.tight_layout()
    plt.savefig(frame_path)
    plt.close(fig)

# Создание каталога для сохранения кадров
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Моделирование
positions = []
for step in range(nstep):
    print(f"STEP: {step}")
    F = compute_forces(x, fixed)  # Рассчитываем силы
    a = F / m  # Вычисляем ускорения
    # Метод Верле
    x_next = 2 * x - x_prev + a * dt**2
    v_new = (x_next - x_prev) / (2 * dt)
    x_prev = x.copy()
    x = x_next.copy()
    v = v_new.copy()
    # Применяем проекционный метод
    enforce_constraints(x)
    positions.append(x.copy())
    # Визуализация
    visualize_step(x, step)


print("Моделирование завершено. Кадры сохранены в каталог 'frames'.")

STEP: 0
STEP: 1
STEP: 2
STEP: 3
STEP: 4
STEP: 5
STEP: 6
STEP: 7
STEP: 8
STEP: 9
STEP: 10
STEP: 11
STEP: 12
STEP: 13
STEP: 14
STEP: 15
STEP: 16
STEP: 17
STEP: 18
STEP: 19
STEP: 20
STEP: 21
STEP: 22
STEP: 23
STEP: 24
Моделирование завершено. Кадры сохранены в каталог 'frames'.


In [3]:
import imageio
import os

N = 1

# Путь к каталогу с кадрами
output_dir = "frames"

# Список файлов кадров
frame_files = sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".png")])

# Создание анимации
with imageio.get_writer(f"animation{N}.gif", mode="I", duration=0.05) as writer:
    for frame_file in frame_files:
        image = imageio.imread(frame_file)
        writer.append_data(image)

print(f"Анимация сохранена как 'animation{N}.gif'.")

/var/folders/wg/vzdjz4_n2q3gtl63x660zg2r0000gn/T/ipykernel_42870/212935973.py:15: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(frame_file)


Анимация сохранена как 'animation1.gif'.
